<a href="https://colab.research.google.com/github/MiguelSix/datasets/blob/main/limpieza_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONFIGURACION DE RUTA Y DATASET


In [ ]:
#MONTAR DRIVE COMPARTIDO

from google.colab import drive
import os

drive.mount('/content/drive')

shared_folder_id = "1ROZfrhsFxeazxoewZVlCydXeMTqQbdLw"

!gdown --folder --id {shared_folder_id}

#!ls -la

Mounted at /content/drive
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Processing file 1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He ASEGURADO.csv
Processing file 1c7eg6e0Q8bL1U6pjrNOjiL9FsVPAN1n6 Aseguradora
Processing file 10puKqkdp7nmBa3DJkbgo6lyNILIfPQhy Dataset.csv
Processing file 1v9xwPresWaRaZ9X-o-I-ohxYHuobDtWLk44Nszn3Qo4 Limpieza de dataset
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He
From (redirected): https://drive.google.com/uc?id=1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He&confirm=t&uuid=fda7559f-588b-4b74-9d7d-5081703edb9f
To: /content/Qualitas/ASEGURADO.csv
100% 633M/633M [00:04<00:00, 138MB/s]
Downloading...
From: https://drive.google.com

In [ ]:
#RUTA/CARGAR ARCHIVO
import pandas as pd

file_path = './Qualitas/ASEGURADO.csv'

df_principal = pd.read_csv(file_path)
print(df_principal.head())

                                  ASEG              SERIE    TELEFONO  \
0              REY DAVID BELMAN LOZANO  9BWKL45U6RP023988  4444191129   
1                  DAVILA MORENO CELSO  3G1TA5AF7HL120346  5560767580   
2        GARCIA GONZALEZ JESUS EDUARDO  1C4GJ25R23B244434  4611366507   
3  SD INDEVAL INSTITUCION PARA EL DEPO  JM3KKKHC2R1166032         NaN   
4           MARIA SEDELIA VARGAS ROSAS  MB2C22AC8JM241475  7531117734   

           DIR_ASEG_CALLE             COLONIA  \
0      CIRCUITO JACARANDA           EL ZAPOTE   
1  1RA CDA FERROCARRIL 12  SAN JUAN TEMAMATLA   
2    PROLONGACION CONSEJO             METEPEC   
3     PASEO DE LA REFORMA          CUAUHTEMOC   
4       LOC EL PUERTECITO           EL REPARO   

                                   POBL            ESTADO       CP  \
0       035 SOLEDAD DE GRACIANO SANCHEZ   SAN LUIS POTOSI  78430.0   
1                         083 TEMAMATLA  ESTADO DE MEXICO  56650.0   
2                           019 ATLIXCO            PUEBL

# ***CODIGOS DE LIMPIEZA DE MIGUEL***

In [ ]:
import unicodedata
import re

##***Columna del número de serie (SERIE)***

**Función para revisar la columna y checar los registros erroneos**

In [ ]:
def analizar_columna_serie(df, columna='SERIE'):

    def _verificar_serie(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor == '':
            return {
                'original': valor,
                'valido': True,
                'problemas': []
            }

        # Convertir a string para asegurar consistencia
        valor = str(valor)

        problemas = []

        # Verificar espacios
        if valor != valor.strip():
            problemas.append('Espacios al inicio/final')

        # Verificar espacios entre caracteres
        if ' ' in valor:
            problemas.append('Espacios entre caracteres')

        # Verificar mayúsculas
        if valor != valor.upper():
            problemas.append('No está en mayúsculas')

        # Verificar caracteres especiales y acentos
        # Primero, remover acentos
        valor_sin_acentos = ''.join(
            c for c in unicodedata.normalize('NFD', valor)
            if unicodedata.category(c) != 'Mn'
        )

        # Verificar solo alfanuméricos
        if not re.match(r'^[A-Z0-9]+$', valor_sin_acentos):
            problemas.append('Contiene caracteres especiales o acentos')

        return {
            'original': valor,
            'valido': len(problemas) == 0,
            'problemas': problemas
        }

    # Aplicar análisis a toda la columna
    analisis = df[columna].apply(_verificar_serie)

    # Resumen estadístico
    resumen = {
        'total_registros': len(df),
        'registros_validos': sum(1 for x in analisis if x['valido']),
        'registros_invalidos': sum(1 for x in analisis if not x['valido']),
        'tipos_de_problemas': {}
    }

    # Contar tipos de problemas
    for item in analisis:
        if not item['valido']:
            for problema in item['problemas']:
                resumen['tipos_de_problemas'][problema] = resumen['tipos_de_problemas'].get(problema, 0) + 1

    return {
        'resumen': resumen,
        'detalles': analisis
    }

**Función para limpiar la columna SERIE (Corregir los errores)**

In [ ]:
def limpiar_columna_serie(df, columna='SERIE'):

    def _limpiar_serie(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor == '':
            return ''

        # Convertir a string
        valor = str(valor)

        # Eliminar acentos
        valor_sin_acentos = ''.join(
            c for c in unicodedata.normalize('NFD', valor)
            if unicodedata.category(c) != 'Mn'
        )

        # Convertir a mayúsculas y eliminar espacios
        valor_limpio = valor_sin_acentos.upper().replace(' ', '')

        # Verificar que solo contenga letras y números
        if re.match(r'^[A-Z0-9]+$', valor_limpio):
            return valor_limpio

        # Si no cumple, devolver cadena vacía
        return ''

    # Crear una copia del DataFrame para no modificar el original
    df_limpio = df.copy()

    # Aplicar limpieza
    df_limpio[columna] = df_limpio[columna].apply(_limpiar_serie)

    return df_limpio

##***Columna del número de telefono (TELEFONO)***

**Función para analizar la columna**

In [ ]:
def analizar_telefonos(df, columna='TELEFONO'):
    # Convertir a string y crear copia para análisis
    serie = df[columna].astype(str)
    total_registros = len(serie)

    # Inicializar contadores
    analisis = {
        'total_registros': total_registros,
        'registros_vacios': 0,
        'mas_10_digitos': 0,
        'tiene_simbolos': 0,
        'tiene_espacios_final': 0,
        'tiene_letras': 0,
        'ejemplos_problematicos': []
    }

    patrones = {
        'mas_10_digitos': [],
        'simbolos': [],
        'espacios_final': [],
        'letras': []
    }

    for valor in serie:
        # Saltear valores nulos o 'nan'
        if pd.isna(valor) or valor == 'nan':
            analisis['registros_vacios'] += 1
            continue

        # Verificar longitud de dígitos
        digitos = re.findall(r'\d', valor)
        if len(digitos) > 10:
            analisis['mas_10_digitos'] += 1
            if len(patrones['mas_10_digitos']) < 3:
                patrones['mas_10_digitos'].append(valor)

        # Verificar símbolos
        if re.search(r'[+()/-]', valor):
            analisis['tiene_simbolos'] += 1
            if len(patrones['simbolos']) < 3:
                patrones['simbolos'].append(valor)

        # Verificar espacios al final
        if valor.endswith(' '):
            analisis['tiene_espacios_final'] += 1
            if len(patrones['espacios_final']) < 3:
                patrones['espacios_final'].append(valor)

        # Verificar letras
        if re.search(r'[a-zA-Z]', valor):
            analisis['tiene_letras'] += 1
            if len(patrones['letras']) < 3:
                patrones['letras'].append(valor)

    # Calcular porcentajes
    for key in ['registros_vacios', 'mas_10_digitos', 'tiene_simbolos', 'tiene_espacios_final', 'tiene_letras']:
        porcentaje = (analisis[key] / total_registros) * 100
        print(f"{key}: {analisis[key]} registros ({porcentaje:.2f}%)")

    print("\nEjemplos de registros problemáticos:")
    for tipo, ejemplos in patrones.items():
        if ejemplos:
            print(f"\n{tipo}:")
            for ejemplo in ejemplos:
                print(f"  - {ejemplo}")

**Funcion para limpiar la columna (corregir errores)**

In [ ]:
def limpiar_solo_numericos(df, columna='TELEFONO'):
    # Convertir a string
    df[columna] = df[columna].astype(str)

    # Función para limpiar cada valor
    def limpiar_valor(valor):
        if pd.isna(valor) or valor == 'nan':
            return ''
        # Extraer solo los números usando expresiones regulares
        numeros = re.sub(r'[^0-9]', '', valor)
        return numeros

    # Aplicar la limpieza
    df[columna] = df[columna].apply(limpiar_valor)

    # Verificar los resultados
    muestra = df[columna].head(10)
    print("\nMuestra de los primeros 10 registros limpiados:")
    for idx, valor in muestra.items():
        print(f"Original: {data[columna][idx]} -> Limpio: {valor}")

    return df

# CODIGOS DE LIMPIEZA
CARLOS

In [ ]:
##CARLOS

# CODIGOS DE LIMPIEZA
RICARDO

In [ ]:
##RICARDO

In [ ]:
#import pandas as pd

# Supongamos que ya cargaste tu DataFrame en 'df_principal'
# df_principal = pd.read_csv('tu_archivo.csv')

# Verificar la columna 'RFC_Aseg'
df_principal['RFC_ASEG'] = df_principal['RFC_ASEG'].apply(
    lambda rfc: rfc.strip().upper() if isinstance(rfc, str) and rfc.strip().isalnum() else None
)

# Mostrar algunas filas del DataFrame para verificar resultados
print("Primeras filas después de la limpieza:")
print(df_principal.head())

# Mostrar filas con errores
errores = df_principal[df_principal['RFC_ASEG'].isnull()]
if not errores.empty:
    print("\nFilas con errores:")
    print(errores)

# Guardar el resultado en un archivo si lo necesitas
#df_principal.to_csv('df_principal_limpio.csv', index=False)
#print("\nArchivo limpio guardado como 'df_principal_limpio.csv'")


Primeras filas después de la limpieza:
                                  ASEG              SERIE    TELEFONO  \
0              REY DAVID BELMAN LOZANO  9BWKL45U6RP023988  4444191129   
1                  DAVILA MORENO CELSO  3G1TA5AF7HL120346  5560767580   
2        GARCIA GONZALEZ JESUS EDUARDO  1C4GJ25R23B244434  4611366507   
3  SD INDEVAL INSTITUCION PARA EL DEPO  JM3KKKHC2R1166032         NaN   
4           MARIA SEDELIA VARGAS ROSAS  MB2C22AC8JM241475  7531117734   

           DIR_ASEG_CALLE             COLONIA  \
0      CIRCUITO JACARANDA           EL ZAPOTE   
1  1RA CDA FERROCARRIL 12  SAN JUAN TEMAMATLA   
2    PROLONGACION CONSEJO             METEPEC   
3     PASEO DE LA REFORMA          CUAUHTEMOC   
4       LOC EL PUERTECITO           EL REPARO   

                                   POBL            ESTADO       CP  \
0       035 SOLEDAD DE GRACIANO SANCHEZ   SAN LUIS POTOSI  78430.0   
1                         083 TEMAMATLA  ESTADO DE MEXICO  56650.0   
2                

# CODIGOS DE LIMPIEZA
YESS

In [ ]:
##YESS

In [ ]:
##validacion CP_CONTRATANTE

import re

def validar_cp_cn(df_principal, columna='CP_CONTRATANTE'):
    valores_invalidos = []

    def es_valido(valor):
        if pd.isna(valor):
            valores_invalidos.append("NaN")
            return False
        valor = str(valor).strip()
        if re.fullmatch(r'^\d{4,5}$', valor):  # Verificar que sea un número de 4 o 5 dígitos
            return True
        else:
            valores_invalidos.append(valor)  # Agregar a los inválidos
            return False

    # Aplicar la validación
    df_principal['es_valido'] = df_principal[columna].apply(es_valido)

    # Filtrar los valores inválidos únicos
    valores_invalidos_unicos = list(set(valores_invalidos))

    # Mostrar resultados
    print(f"Total de valores inválidos: {len(valores_invalidos)}")
    print("Valores inválidos únicos encontrados:")
    for valor in valores_invalidos_unicos:
        print(f"- {valor}")

    # Retornar el DataFrame actualizado y los valores inválidos
    return df_, valores_invalidos_unicos

df_actualizado, valores_invalidos = validar_cp_cn(df, columna='cp_cn')


# GUARDAR ARCHIVO MODIFICADO

In [ ]:
# Ruta para guardar el archivo consolidado
output_csv_path = '/content/drive/My Drive/Qualitas/Dataset.csv'

# Guardar el DataFrame final
df_principal.to_csv(output_csv_path, index=False)

print(f"Archivo consolidado guardado en: {output_csv_path}")


Archivo consolidado guardado en: /content/drive/My Drive/Qualitas/Dataset.csv
